## 1. 환경 설정

In [1]:
# TensorFlow 모델 레포지토리를 GitHub에서 클론
# (TensorFlow Object Detection API 설치를 위해 필요)
!pip uninstall Cython -y  # "No module named 'object_detection'" 오류를 임시로 해결하기 위해 Cython을 제거
!git clone --depth 1 https://github.com/tensorflow/models  # TensorFlow 모델 소스 코드를 가져옴 (최신 커밋만 클론)

Found existing installation: Cython 3.0.11
Uninstalling Cython-3.0.11:
  Successfully uninstalled Cython-3.0.11
Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3315/3315), done.
remote: Total 4305 (delta 1212), reused 2205 (delta 917), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.16 MiB | 9.10 MiB/s, done.
Resolving deltas: 100% (1212/1212), done.
Updating files: 100% (3875/3875), done.


In [2]:
# 모델 설정 파일을 models/research 폴더로 복사 및 프로토콜 버퍼 파일 컴파일

# Bash 셸 명령어 실행
%%bash
cd models/research/

# Object Detection API에서 사용하는 .proto 파일을 컴파일하여 Python 코드로 변환
protoc object_detection/protos/*.proto --python_out=.

In [3]:
# setup.py 파일 수정: TensorFlow 모델 레포지토리를 TF v2.8.0에 맞춰 설치
import re  # 정규 표현식을 사용하기 위한 모듈

# 기존 setup.py 파일의 내용을 읽어옴
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()  # setup.py 파일 내용을 문자열로 읽기

# 수정된 내용을 새로운 setup.py 파일에 작성
with open('/content/models/research/setup.py', 'w') as f:
    # "tf-models-official>=2.5.1" 문자열을 "tf-models-official==2.8.0"으로 변경
    # TensorFlow 버전 호환성을 위해 특정 버전으로 고정
    s = re.sub('tf-models-official>=2.5.1', 'tf-models-official==2.8.0', s)
    f.write(s)  # 수정된 내용을 새로운 setup.py 파일에 저장


- 아래 셀 실행 시 발생하는 **의존성 오류는 무시**
  
- 중간에 '**세션 다시 시작**' 팝업이 뜰 경우  
세션 다시 시작 클릭 -> 아래 셀을 한 번 더 실행

In [ ]:
# Object Detection API 설치 (참고: 이 코드 블록 실행에 약 10분 정도 소요될 수 있음)

# PyYAML 라이브러리 관련 문제 해결
# Colab 환경에서 PyYAML v5.4.1을 설치할 수 없으므로 PyYAML v5.3 버전을 설치
!pip install pyyaml==5.3

# Object Detection API를 설치
# 연구(research) 폴더에 있는 Python 패키지를 설치하여 사용할 준비
!pip install /content/models/research/

# TensorFlow 버전을 2.8.0으로 다운그레이드
# Colab 환경에서 TensorFlow v2.10과의 호환성 문제(2022년 10월 기준)가 있어 v2.8.0으로 변경
!pip install tensorflow==2.8.0

# TensorFlow v2.8.0과 호환되는 CUDA v11.0 설치
# TensorFlow v2.8.0은 CUDA v11.0과 호환되므로 이를 설치하여 GPU 가속을 지원
!pip install tensorflow_io==0.23.1  # TensorFlow와 호환되는 I/O 라이브러리 설치

# CUDA v11.0 관련 패키지를 다운로드하고 설치
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600

# CUDA v11.0 설치 파일을 다운로드
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

# CUDA 설치 파일을 dpkg 명령어로 설치
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

# CUDA의 공개 키를 추가하여 패키지를 인증
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub

# 시스템 패키지 업데이트 및 CUDA 툴킷 v11.0 설치
!apt-get update && sudo apt-get install cuda-toolkit-11-0

# CUDA 라이브러리 경로를 환경 변수에 추가
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

Processing ./models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697355 sha256=fe69c36978ae9b262d6916f63d833e2c9cb35ddd00038afe6ef10ca2bd20b35b
  Stored in directory: /tmp/pip-ephem-wheel-cache-zyctofz9/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1
  Using cached tensorflow-2.8.0-cp310-cp310-manylinux2010_x86_64.whl.metadata (2.9 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: te

In [ ]:
# Protobuf 패키지 문제 해결을 위한 설치 과정

!pip uninstall -y protobuf  # Protobuf 패키지를 제거 (-y 옵션으로 사용자 확인 없이 제거)
!pip install protobuf==3.20.1  # Protobuf 버전을 3.20.1로 다시 설치

Found existing installation: protobuf 4.25.5
Uninstalling protobuf-4.25.5:
  Successfully uninstalled protobuf-4.25.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.61.0 requires protobuf!=4.0.*,!=4.21.*,!=4.22.0,!=4.23.*,!=4.24.*,<6.0.0.dev0,>=3.20.3, but you have protobuf 3.20.1 which is incompatible.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.19.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-aiplatform 1.71.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev

In [ ]:
import os

# 현재 실행 중인 프로세스를 강제로 종료
os.kill(os.getpid(), 9)  # os.getpid()로 현재 프로세스 ID를 가져오고, 신호 9(SIGKILL)로 해당 프로세스를 종료

In [ ]:
# Protobuf 패키지 정보 확인
!pip show protobuf

Name: protobuf
Version: 3.20.1
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: apache-beam, google-ai-generativelanguage, google-api-core, google-cloud-aiplatform, google-cloud-bigquery-connection, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-datastore, google-cloud-firestore, google-cloud-functions, google-cloud-iam, google-cloud-language, google-cloud-pubsub, google-cloud-resource-manager, google-cloud-translate, google-generativeai, googleapis-common-protos, grpc-google-iam-v1, grpcio-status, orbax-checkpoint, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata, wandb


In [ ]:
# 모델 빌더 테스트 파일 실행: 설정이 올바르게 작동하는지 확인하기 위한 테스트

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py
# Object Detection API의 모델 빌더 구성 요소를 테스트하는 스크립트를 실행
# 이 스크립트는 설치 및 구성 파일들이 제대로 작동하는지 확인하는 데 사용됨

2024-11-30 09:32:02.674506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-11-30 09:32:02.674591: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W1130 09:32:03.655683 137511117709312 model_builder.py:1112] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.91s
I1130 09:32:05.594129 137511117709312 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.91s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_mod

## 2. 경로 설정

In [ ]:
# # 객체 탐지 모델이 탐지할 클래스 목록을 포함하는 "labelmap.pdtxt" 파일 생성
# %%bash

# # labelmap.txt 파일에 클래스 이름을 추가
# cat <<EOF >> /content/labelmap.pdtxt
# item {
#     name: "0 motorcycle",
#     id: 1,
#     display_name: "0 motorcycle"
# }
# item {
#     name: "1 bicycle",
#     id: 2,
#     display_name: "1 bicycle"
# }
# item {
#     name: "2 kickboard",
#     id: 3,
#     display_name: "2 kickboard"
# }
# EOF

In [ ]:
from google.colab import drive

# Google Drive 연결
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 경로 설정
saved_model_dir = "/content/drive/MyDrive/nipa_google/model"  # 저장된 모델 경로
tfrecord_path = "/content/drive/MyDrive/nipa_google/data/my_cam.v3i.tfrecord/train/bicycle-motorcycle-kickboard.tfrecord"  # 테스트용 TFRecord 파일 경로
label_map_path = "/content/drive/MyDrive/nipa_google/data/my_cam.v3i.tfrecord/train/bicycle-motorcycle-kickboard_label_map.pbtxt"  # 클래스 라벨 맵 파일 경로

In [ ]:
model_path_list = os.listdir(saved_model_dir)

## 3. 성능 평가 & 추론 속도 측정

In [ ]:
import tensorflow as tf
import numpy as np
from object_detection.utils import metrics
from object_detection.utils import np_box_ops
from object_detection.utils import label_map_util
from tqdm import tqdm
import time


# 모델 로드
model = tf.saved_model.load(saved_model_dir)
infer = model.signatures["serving_default"]

# 라벨 맵 로드 (클래스 ID와 이름 매핑)
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)


In [ ]:
# TFRecord 데이터 로드 함수
def parse_tfrecord_fn(record):
    """TFRecord 데이터를 파싱하는 함수."""
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(record, feature_description)

    # 이미지 디코딩 및 uint8 형식으로 변환
    image = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    image = tf.image.resize(image, (320, 320))  # 모델 입력 크기에 맞게 리사이즈
    image = tf.cast(image, tf.uint8)  # uint8로 변환

    # 정답 박스와 클래스 추출
    xmin = tf.sparse.to_dense(example['image/object/bbox/xmin'])
    xmax = tf.sparse.to_dense(example['image/object/bbox/xmax'])
    ymin = tf.sparse.to_dense(example['image/object/bbox/ymin'])
    ymax = tf.sparse.to_dense(example['image/object/bbox/ymax'])
    classes = tf.sparse.to_dense(example['image/object/class/label'])

    boxes = tf.stack([ymin, xmin, ymax, xmax], axis=-1)  # [ymin, xmin, ymax, xmax]
    return image, boxes, classes

# TFRecord 데이터셋 로드
raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
parsed_dataset = parsed_dataset.batch(1)  # 배치 크기 설정

# 평가를 위한 데이터 저장
all_detections = []
all_ground_truths = []
inference_times = []  # 추론 시간 저장

print("\n--- Inference Times per Image ---")
for image, gt_boxes, gt_classes in tqdm(parsed_dataset):
    # 모델 추론 시간 측정
    start_time = time.time()
    output = infer(tf.convert_to_tensor(image))
    end_time = time.time()

    inference_time = end_time - start_time
    inference_times.append(inference_time)

    print(f"Inference Time: {inference_time:.4f} seconds")

    # 모델 예측 결과
    detection_boxes = output["detection_boxes"].numpy()[0]
    detection_scores = output["detection_scores"].numpy()[0]
    detection_classes = output["detection_classes"].numpy()[0].astype(np.int32)

    # 신뢰도 기준 필터링
    valid_detections = detection_scores > 0.5
    detection_boxes = detection_boxes[valid_detections]
    detection_scores = detection_scores[valid_detections]
    detection_classes = detection_classes[valid_detections]

    # 저장
    all_detections.append((detection_boxes, detection_scores, detection_classes))
    all_ground_truths.append((gt_boxes.numpy(), gt_classes.numpy()))

# 평균 추론 시간 계산
average_inference_time = sum(inference_times) / len(inference_times)
print(f"\n--- Average Inference Time ---")
print(f"Average Time per Image: {average_inference_time:.4f} seconds")

# mAP, Precision, Recall 계산
iou_threshold = 0.5  # IoU 임계값

average_precisions = []
precisions = []
recalls = []

for i in range(len(all_detections)):
    det_boxes, det_scores, det_classes = all_detections[i]
    gt_boxes, gt_classes = all_ground_truths[i]

    if len(gt_boxes) == 0:
        # Ground Truth가 없을 경우 스킵
        continue

    # 클래스별로 Precision/Recall 계산
    for class_id in np.unique(gt_classes):
        gt_mask = gt_classes == class_id
        det_mask = det_classes == class_id

        # 클래스별 Ground Truth 및 Detection 필터링
        gt_boxes_class = gt_boxes[gt_mask]
        det_boxes_class = det_boxes[det_mask]
        det_scores_class = det_scores[det_mask]

        # IoU 계산
        iou_matrix = np_box_ops.iou(det_boxes_class, gt_boxes_class)
        true_positive = (iou_matrix > iou_threshold).sum()
        false_positive = len(det_boxes_class) - true_positive
        false_negative = len(gt_boxes_class) - true_positive

        # Precision, Recall 계산
        precision = true_positive / (true_positive + false_positive + 1e-8)
        recall = true_positive / (true_positive + false_negative + 1e-8)

        precisions.append(precision)
        recalls.append(recall)

    # AP 계산
    ap = sum(precisions) / len(precisions) if len(precisions) > 0 else 0
    average_precisions.append(ap)

# 전체 mAP, Precision, Recall
mean_ap = np.mean(average_precisions) if len(average_precisions) > 0 else 0
mean_precision = np.mean(precisions) if len(precisions) > 0 else 0
mean_recall = np.mean(recalls) if len(recalls) > 0 else 0

print()
print(f"Mean Average Precision (mAP): {mean_ap}")
print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")

41it [00:12,  3.30it/s]


Mean Average Precision (mAP): 0.0
Mean Precision: 0.0
Mean Recall: 0.0


## 4. 바운딩 박스 확인

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt


# 모델 경로 및 이미지 폴더 경로 설정
MODEL_PATH = "/content/drive/MyDrive/nipa_google/model/saved_model"  # 저장된 모델 경로
IMAGE_DIR = "/content/drive/MyDrive/nipa_google/data/data_cam"  # 이미지 폴더 경로


# 모델 로드
detect_fn = tf.saved_model.load(MODEL_PATH)

def draw_bounding_boxes(image, boxes, scores, classes, threshold=0.5):
    """
    이미지에 바운딩 박스를 그림.
    Args:
        image: 입력 이미지 (numpy 배열).
        boxes: 바운딩 박스 좌표 (ymin, xmin, ymax, xmax) [N, 4].
        scores: 박스 신뢰도 점수 [N].
        classes: 클래스 ID [N].
        threshold: 점수 임계값.
    """
    for i in range(len(boxes)):
        if scores[i] < threshold:
            continue

        # 바운딩 박스 좌표
        ymin, xmin, ymax, xmax = boxes[i]
        start_point = (int(xmin * image.shape[1]), int(ymin * image.shape[0]))
        end_point = (int(xmax * image.shape[1]), int(ymax * image.shape[0]))

        # 바운딩 박스 색상 및 두께
        color = (0, 255, 0)  # 초록색
        thickness = 2

        # 바운딩 박스 그리기
        image = cv2.rectangle(image, start_point, end_point, color, thickness)

        # 클래스 및 점수 표시 (폰트 크기와 두께 조정)
        label = f"Class {int(classes[i])}: {scores[i]:.2f}"
        font_scale = 1.2  # 폰트 크기 증가
        font_thickness = 3  # 글자 굵기 증가
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        text_origin = (start_point[0], start_point[1] - 10 if start_point[1] - 10 > 10 else start_point[1] + text_size[1])

        cv2.putText(
            image,
            label,
            text_origin,
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            color,
            font_thickness,
        )
    return image

# 이미지 폴더 내 모든 파일 처리
for file_name in os.listdir(IMAGE_DIR):
    file_path = os.path.join(IMAGE_DIR, file_name)

    # 이미지 읽기
    image = cv2.imread(file_path)
    if image is None:
        print(f"Skipping {file_name}, not an image.")
        continue

    # 이미지 전처리: 320x320으로 리사이즈 및 uint8 변환
    input_tensor = cv2.resize(image, (320, 320))  # 리사이즈
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]  # 배치 차원 추가

    # 모델 추론
    detections = detect_fn(input_tensor)

    # 바운딩 박스 정보 추출
    detection_boxes = detections['detection_boxes'].numpy()[0]  # [N, 4]
    detection_scores = detections['detection_scores'].numpy()[0]  # [N]
    detection_classes = detections['detection_classes'].numpy()[0].astype(np.int32)  # [N]

    # 바운딩 박스 그리기
    result_image = draw_bounding_boxes(image, detection_boxes, detection_scores, detection_classes)

    # 이미지 출력
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(file_name)
    plt.show()